In [54]:
import pandas as pd
import re
import numpy as np
from sympy.solvers import solve
from sympy import Symbol, simplify
from sympy import sieve
from random import choice

In [55]:
table = pd.read_csv('dialog/patterns/artith_qa.csv').fillna('')

In [4]:
def parse_line(line):
    steps = list(line[['r1', 'r2', 'r3', 'r4', 'r5']].values)
    reqs = list(line[['r1_req', 'r2_req', 'r3_req', 'r4_req', 'r5_req']].values)
    words = {}
    word_deps = {}
    for var, word in zip(['word1', 'word2', 'word3', 'word4', 'word5', 'word6', 'word7'],
                        line[['word1', 'word2', 'word3', 'word4', 'word5', 'word6', 'word7']].values):
        if '@' in word:
            dep, options = word.split('@')
            word_deps[var] = dep
        else:
            options = word
        
        options = [w.replace('\\', '').split(';') for w in split_esc(options, ',')]
        words[var] = options
    formula = line['formula']   
    ranges = {}
    for v in line['var_range'].split(';'):
        var, rang = v.split('=')
        ranges[var] = eval(rang)
    context = line['context']
    question = line['q']
    defined_vars = {}
    if line['defined_vars']:
        
    
        for v in line['defined_vars'].split(';'):
            var, val = v.split('=')
            defined_vars[var] = int(val)
    
    return context, question, steps, reqs, ranges, word_deps, words, defined_vars

In [5]:
import math
def divisors(n, rang=None):
    if not rang:
        rang= (2, int(math.sqrt(n))+1)
    else:
        rang = (max(rang[0], 2), int(math.sqrt(rang[1]))+1)
    divs = set()
    for i in range(*rang):
        if n%i == 0:
            divs.update([i])
            if n/i < rang[1]:
                divs.update([n/i])
            
    
    return list(set(divs))

In [6]:
def generate_numbers_add_or_mult(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i):
    if '+' in step:
        variables = step.split('+')
    elif '*' in step:
        variables = step.split('*')
    else:
        raise KeyError

    
    #нет условия сверху, наверное все через интервалы переменных задать можно
    # но надо будет проверять
    if req:
        if variables[0] not in defined_vars and variables[1] not in defined_vars:
            v1 = np.random.randint(*ranges[variables[0]])
            exec(f'{variables[0]} = {v1}')
            exec(f'{variables[1]} = Symbol("{variables[1]}")')
            exec(f'r{i+1}= {step}')
            sol = solve((eval(req)).subs(variables[0], v1), variables[1])

            sol = sol.as_set()
            min_sol, max_sol = float(sol.left), float(sol.right)
            min_rang, max_rang = ranges[variables[1]]

            v2 = np.random.randint(max([min_sol, min_rang]), 
                                       min([max_sol, max_rang]))
            
        elif variables[0] in defined_vars:
            v1 = defined_vars[variables[0]]
            exec(f'{variables[1]} = Symbol("{variables[1]}")')
            exec(f'{variables[0]} = {v1}')
            exec(f'r{i+1}= {step}')
            sol = solve((eval(req)).subs(variables[0], v1), variables[1])
#             print(req)
#             print((eval(req)))
            sol = sol.as_set()
            min_sol, max_sol = float(sol.left), float(sol.right)
            min_rang, max_rang = ranges[variables[1]]

            v2 = np.random.randint(max([min_sol, min_rang]), 
                                       min([max_sol, max_rang]))
            
        elif variables[1] in defined_vars:
            v2 = defined_vars[variables[1]]
            exec(f'{variables[1]} = {v2}')
            exec(f'{variables[0]} = Symbol("{variables[0]}")')
            exec(f'r{i+1}= {step}')
            sol = solve((eval(req)).subs(variables[1], v2), variables[0])

            sol = sol.as_set()
            min_sol, max_sol = float(sol.left), float(sol.right)
            min_rang, max_rang = ranges[variables[0]]

            v1 = np.random.randint(max([min_sol, min_rang]), 
                                       min([max_sol, max_rang]))
            
    else:

        v1 = defined_vars.get(variables[0]) or np.random.randint(*ranges[variables[0]])
        v2 = defined_vars.get(variables[1]) or np.random.randint(*ranges[variables[1]])
    
    exec(f'{variables[0]} = {v1}')
    exec(f'{variables[1]} = {v2}')
    result = eval(step)
    
    if f'r{i+1}' in not_prime_req and result in sieve:
        return generate_numbers_add_or_mult(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
    elif f'r{i+1}' in res_dividable_req:
        if res_dividable_req[f'r{i+1}']%result:
            return generate_numbers_add_or_mult(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
        else:
            return result
    else:
        return v1, v2, result



In [7]:
def generate_numbers_subst(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i):
    variables = step.split('-')
    

    
    #условия снизу плохо учитываются. Если в r1>100, а первое число 
    # в интервале 1,300, наверное все обходится через правильные интервалы
    # или если r1<1000 , v1=5000, а у второго  интервал меньше 4 к
    if req:
        if variables[0] not in defined_vars and variables[1] not in defined_vars:
            v1 = np.random.randint(*ranges[variables[0]])
            exec(f'{variables[0]} = {v1}')
            exec(f'{variables[1]} = Symbol("{variables[1]}")')
            exec(f'r{i+1}= {step}')
            sol = solve((eval(req)).subs(variables[0], v1), variables[1])

            sol = sol.as_set()
            min_sol, max_sol = float(sol.left), float(sol.right)
            min_rang, max_rang = ranges[variables[1]]
            min_r, max_r = (max([min_sol, min_rang]), 
                                       min([max_sol, max_rang, v1-1]))
#             if min_r >= max_r:
                
#                 return generate_numbers_subst(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
            
            v2 = np.random.randint(min_r, max_r)
            
        elif variables[0] in defined_vars:
            v1 = defined_vars[variables[0]]
            exec(f'{variables[1]} = Symbol("{variables[1]}")')
            exec(f'{variables[0]} = {v1}')
            exec(f'r{i+1}= {step}')
            sol = solve((eval(req)), variables[1])
#             print(req)
#             print((eval(req)))
            sol = sol.as_set()
            min_sol, max_sol = float(sol.left), float(sol.right)
            min_rang, max_rang = ranges[variables[1]]
            min_r, max_r = (max([min_sol, min_rang]), 
                                       min([max_sol, max_rang, v1-1]))
#             if min_r >= max_r:
#                 print(v1, min_r, max_r)
#                 return generate_numbers_subst(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
            
            v2 = np.random.randint(min_r, max_r)
            
        elif variables[1] in defined_vars:
            v2 = defined_vars[variables[1]]
            exec(f'{variables[1]} = {v2}')
            exec(f'{variables[0]} = Symbol("{variables[0]}")')
            exec(f'r{i+1}= {step}')
            sol = solve((eval(req)), variables[0])

            sol = sol.as_set()
            min_sol, max_sol = float(sol.left), float(sol.right)
            min_rang, max_rang = ranges[variables[0]]
            min_r, max_r = (max([v2+1, min_sol, min_rang]), 
                                       min([max_sol, max_rang]))
#             if min_r >= max_r:
#                 return generate_numbers_subst(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
            
            v1 = np.random.randint(min_r, max_r)
            
    else:
        # допилить для определенных r1 
        v1 = defined_vars.get(variables[0]) or np.random.randint(*ranges[variables[0]])
        min_r, max_r = ranges[variables[1]]
        v2 = defined_vars.get(variables[1]) or np.random.randint(min_r, min(max_r, v1))
    
    exec(f'{variables[0]} = {v1}')
    exec(f'{variables[1]} = {v2}')
    result = eval(step)
    
    if f'r{i+1}' in not_prime_req and result in sieve:
        return generate_numbers_subst(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
    elif f'r{i+1}' in res_dividable_req:
        if res_dividable_req[f'r{i+1}']%result:
            return generate_numbers_subst(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
        else:
            return result
    elif result < 1:
        return generate_numbers_subst(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
    else:
        return v1, v2, result



In [8]:
def generate_numbers_div(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i):
    variables = step.split('/')
    

    
    #пока не работает
    # нужно как-то учитывать условия
    # ещё продумать что делать с определенным вторым аргументов
    # делители при определенном втором аргументе могут не найтись и будут ошибки
    # но наверное лучше оставить так, потому что заново ничего уже не сгенеришь
    # если ошибки будут часто, то надо будет продумывать откат назад
    # но пока кажется что требование на делимость будет достаточно
    if req:
        if variables[0] not in defined_vars and variables[1] not in defined_vars:
            v1 = np.random.choice([i for i in range(*ranges[variables[0]]) if i not in sieve])
            exec(f'{variables[0]} = {v1}')
            exec(f'{variables[1]} = Symbol("{variables[1]}")')
            exec(f'r{i+1}= {step}')
            sol = solve((eval(req)).subs(variables[0], v1), variables[1])
            sol = sol.as_set()
            min_sol, max_sol = float(sol.left), float(sol.right)
            min_rang, max_rang = ranges[variables[1]]
            min_r, max_r = (max([min_sol, min_rang]), 
                                       min([max_sol, max_rang]))
            factors = divisors(v1, (round(min_r), round(max_r)+1))
            if not factors:
                return generate_numbers_div(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
            v2 = np.random.choice(factors)
            
        elif variables[0] in defined_vars:
            v1 = defined_vars[variables[0]]
            exec(f'{variables[0]} = {v1}')
            exec(f'{variables[1]} = Symbol("{variables[1]}")')
            exec(f'r{i+1}= {step}')
            sol = solve((eval(req)), variables[1])
            sol = sol.as_set()
            min_sol, max_sol = float(sol.left), float(sol.right)
            min_rang, max_rang = ranges[variables[1]]
            min_r, max_r = (max([min_sol, min_rang]), 
                                       min([max_sol, max_rang]))
            factors = divisors(v1, (round(min_r), round(max_r)+1))
            if not factors:
                return generate_numbers_div(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
            v2 = np.random.choice(factors)
            
        elif variables[1] in defined_vars:
            v2 = defined_vars[variables[1]]
            exec(f'{variables[1]} = {v2}')
            exec(f'{variables[0]} = Symbol("{variables[0]}")')
            exec(f'r{i+1}= {step}')
            sol = solve((eval(req)), variables[0])
            sol = sol.as_set()
            min_sol, max_sol = float(sol.left), float(sol.right)
            min_rang, max_rang = ranges[variables[0]]
            min_r, max_r = (max([min_sol, min_rang]), 
                                       min([max_sol, max_rang]))

            v1 = np.random.choice([i for i in range(round(min_r), round(max_r)+1) if not i%v2 and i!=1])
            
    else:
        if variables[0] not in defined_vars and variables[1] not in defined_vars:
            v1 = np.random.choice([i for i in range(*ranges[variables[0]]) if i not in sieve])

            factors = divisors(v1, ranges[variables[1]])
            if not factors:
                return generate_numbers_div(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
            v2 = np.random.choice(factors)
        elif variables[0] in defined_vars:
            v1 = defined_vars[variables[0]]

            factors = divisors(v1, ranges[variables[1]])
            if not factors:
                return generate_numbers_div(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
            v2 = np.random.choice(factors)
        elif variables[1] in defined_vars:
            v2 = defined_vars[variables[1]]
            v1 = np.random.choice([i for i in range(*ranges[variables[0]]) if not i%v2 and i!=1])
            
        
    exec(f'{variables[0]} = {v1}')
    exec(f'{variables[1]} = {v2}')
    result = eval(step)
    
    if f'r{i+1}' in not_prime_req and result in sieve:
        return generate_numbers_div(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
    elif f'r{i+1}' in res_dividable_req:
        if res_dividable_req[f'r{i+1}']%result:
            return generate_numbers_div(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
        else:
            return int(v1), int(v2), int(result)
    elif result < 1:
        return generate_numbers_div(step, req, ranges, defined_vars, not_prime_req, res_dividable_req, i)
    else:
        return int(v1), int(v2), int(result)



In [111]:
(eval(req)).subs(variables[0], v1)

cn2 + 28 > 100

In [352]:
generate_numbers_add_or_mult(step='r1+cn2', req='r2>1000',  defined_vars={'r1':10},
                                 ranges={'cn1': (1, 15000), 'cn2': (1, 13000)}, 
                                 not_prime_req={'cn2',},
                                 res_dividable_req={}, i=1)

(10, 6294, 6304)

In [356]:
generate_numbers_add_or_mult('cn1*cn2', 'r1>1000',   defined_vars={'r1':10},
                             ranges={'cn1': (1, 500), 'cn2': (1, 1300)}, 
                                 not_prime_req={'cn2',},
                                 res_dividable_req={}, i=0)

(131, 231, 30261)

In [357]:
generate_numbers_subst('r1-cn2', 'r2>10', defined_vars={'r1':100}, ranges={'cn1': (1, 15000), 'cn2': (1, 15000)}, 
                                 not_prime_req={'cn2',},
                                 res_dividable_req={}, i=1)

(100, 33, 67)

In [422]:
generate_numbers_div('cn1/cn2', 'r1>3', defined_vars={'cn1':1000}, ranges={'cn1': (10, 5000), 'cn2': (5, 5000)}, 
                                 not_prime_req={'cn2',},
                                 res_dividable_req={}, i=0)

(1000, 5, 200)

In [217]:


v1 = np.random.choice([i for i in range(*ranges[variables[0]]) if i not in sieve])

factors = divisors(v1, ranges[variables[1]])
v2 = np.random.choice(factors)

print(v1, v2, v1/v2)

2615 5.0 523.0


In [341]:
step='r1-cn2'
req='r2<100'
ranges={'cn1': (1, 5000), 'cn2': (1, 1300)}
variables = step.split('-')                              
not_prime_req={'cn2',}
defined_vars = {'r1':100}                 
res_dividable_req={}
i=1

v1 = defined_vars[variables[0]]
exec(f'{variables[1]} = Symbol("{variables[1]}")')
exec(f'{variables[0]} = {v1}')
exec(f'r{i+1}= {step}')
sol = solve((eval(req)), variables[1])
#             print(req)
#             print((eval(req)))
sol = sol.as_set()
min_sol, max_sol = float(sol.left), float(sol.right)
min_rang, max_rang = ranges[variables[1]]
min_r, max_r = (max([min_sol, min_rang]), 
                           min([max_sol, max_rang, v1-1]))


# v1 = np.random.choice([i for i in range(*ranges[variables[0]]) if i not in sieve])
# exec(f'{variables[0]} = {v1}')
# exec(f'{variables[1]} = Symbol("{variables[1]}")')
# exec(f'r{i+1}= {step}')
# sol = solve((eval(req)).subs(variables[0], v1), variables[1])
# sol = sol.as_set()
# min_sol, max_sol = float(sol.left), float(sol.right)
# min_rang, max_rang = ranges[variables[1]]
# min_r, max_r = (max([min_sol, min_rang]), 
#                            min([max_sol, max_rang]))
# factors = divisors(v1, (round(min_r), round(max_r)+1))
# v2 = np.random.choice(factors)
# print(v1, v2, v1/v2)

In [9]:



    


def generate(context, question, steps, reqs, ranges, word_deps, words, defined_vars={}):
    
    
    not_prime_req = set()
    for step in steps:
        if '/' in step and step.split('/')[0].startswith('r'):
            not_prime_req.add(step.split('/')[0])
    res_dividable_req = {}
    for step in steps:
        if '/' in step:
            v1, v2 = step.split('/')
            if v1.startswith('r') and v2.startswith('r'):
                res_dividable_req[v2] = v1
    
    
    
    for i, step in enumerate(steps):

        if '*' in step:
            v1, v2, result = generate_numbers_add_or_mult(step, reqs[i], ranges, 
                                                          defined_vars, not_prime_req, 
                                                          res_dividable_req, i)
            v1_name, v2_name = step.split('*')
            defined_vars[v1_name] = v1
            defined_vars[v2_name] = v2
            defined_vars['result'] = result
            defined_vars[f'r{i+1}'] = result
        elif '+' in step:
            v1, v2, result = generate_numbers_add_or_mult(step, reqs[i], ranges, 
                                                          defined_vars, not_prime_req, 
                                                          res_dividable_req, i)
            v1_name, v2_name = step.split('+')
            defined_vars[v1_name] = v1
            defined_vars[v2_name] = v2
            defined_vars['result'] = result
            defined_vars[f'r{i+1}'] = result
        
        elif '-' in step:
            v1, v2, result = generate_numbers_subst(step, reqs[i], ranges, 
                                                          defined_vars, not_prime_req, 
                                                          res_dividable_req, i)
            v1_name, v2_name = step.split('-')
            defined_vars[v1_name] = v1
            defined_vars[v2_name] = v2
            defined_vars['result'] = result
            defined_vars[f'r{i+1}'] = result
        
        elif '/' in step:
            v1, v2, result = generate_numbers_div(step, reqs[i], ranges, 
                                                          defined_vars, not_prime_req, 
                                                          res_dividable_req, i)
            v1_name, v2_name = step.split('/')
            defined_vars[v1_name] = v1
            defined_vars[v2_name] = v2
            defined_vars['result'] = result
            defined_vars[f'r{i+1}'] = result
    
    intermediate_chosen = {}
    for i in range(1, 8):
        key = f'word{i}'
        value = words[key]

        if len(value) == 1 and value[0][0].startswith('word'):
            # доработать чтобы разрешать зависимости от чисел
            # пока зависит только от числа, но падеж и число одновременно скорее всего тоже нужны
            
            
            wordcase = value[0][0].split('_')
            if len(wordcase) == 1:
                word = wordcase[0]
                if key in word_deps:
                    idx = check_one_ending(str(defined_vars[word_deps[key]]))
                    
                else:
                    idx = -1
                
                defined_vars[key] = intermediate_chosen[word][idx]
                intermediate_chosen[key] = intermediate_chosen[word]
            
            elif wordcase[1] in ['nom', 'gen', 'dat', 'acc']:
                word = wordcase[0]
                case = ['nom', 'gen', 'dat', 'acc'].index(wordcase[1])
            
#             elif wordcase[0] in 
                defined_vars[key] = intermediate_chosen[word][case]
                intermediate_chosen[key] = intermediate_chosen[word]
        
        elif len(value) == 1 and value[0][0].startswith('name'):
            _,gender,case = value[0][0].split('_')
            if gender == 'masc':
                name = choice(names_masc)
#                 name = ['Антон', 'Антона', "Антону"]
            else:
                name = choice(names_fem)
#                 name = ['Вика', 'Вики', "Вике"]
            
            intermediate_chosen[key] = name
            defined_vars[key] = name[['nom', 'gen', 'dat', 'acc'].index(case)]
            
    
        else:
        
            word_forms = choice(value)
            
            if len(word_forms) > 1:
                if key not in word_deps:
                    print(f'Unresolved num dependency for var {key}')
                    raise KeyError
                num = str(defined_vars[word_deps[key]])
                chosen_idx = check_one_ending(num)
                intermediate_chosen[key] = word_forms
                defined_vars[key] = word_forms[chosen_idx]
            else:
                intermediate_chosen[key] = word_forms
                defined_vars[key] = word_forms[0]

      
                    
    
    return context.format(**defined_vars), question.format(**defined_vars), \
            defined_vars['result']

In [10]:
def check_one_ending(num):
    if num[-1] == '1':
        if len(num) == 1 or num[-2] != '1':
            chosen = 0
        else:
            chosen = 2
    elif num[-1] in '234':
        if len(num) == 1 or num[-2] != '1':
            chosen = 1
        else:
            chosen = 2
    else:
        chosen = 2
        
    return chosen

def split_esc(string, delimiter):
    if len(delimiter) != 1:
        raise ValueError('Invalid delimiter: ' + delimiter)
    ln = len(string)
    i = 0
    j = 0
    while j < ln:
        if string[j] == '\\':
            if j + 1 >= ln:
                yield string[i:j]
                return
            j += 1
        elif string[j] == delimiter:
            yield string[i:j]
            i = j + 1
        j += 1
    yield string[i:j]

In [574]:
check_one_ending('32')

1

In [460]:
a = '3'

In [73]:
a = '1,2,3\, 4'

In [74]:
[x.replace('\\', '') for x in split_esc(a, ',')]

['1', '2', '3, 4']

In [592]:
parse_line(table.iloc[2])

('У {word1} было {cn1} {word3}. У {word2} было {cn2} {word4}.',
 'Сколько {word6} {word5}?',
 ['cn1+cn2', '', '', '', ''],
 ['', '', '', '', ''],
 {'cn1': (1, 100), 'cn2': (1, 120)},
 {'word3': 'cn1', 'word4': 'cn2'},
 {'word1': [['name_fem_gen']],
  'word2': [['name_fem_gen']],
  'word3': [['брелок', 'брелка', 'брелков']],
  'word4': [['word3']],
  'word5': [['word3']],
  'word6': [['у них'], ['у них всего'], ['всего'], ['у девочек всего']],
  'word7': [['']]},
 {})

In [438]:
c, q, steps, reqs, ranges, word_deps, words, defined_vars = parse_line(table.iloc[1])

In [681]:
dataset = []
for i in range(table.shape[0]):
    for j in range(1000):
        c, q, r = generate(*parse_line(table.iloc[i]))
        c = c[0].upper() + c[1:]
        q = q[0].upper() + q[1:]
        dataset.append((c, q, r))

In [57]:
generate(*parse_line(table.iloc[5]))

('У Михаила было 7395 рублей. Он дал своему внуку 590 рублей.',
 'Сколько у него после этого осталось денег?',
 6805)

In [682]:
len(set(dataset))

4000

In [571]:
parse_line(table.iloc[2])

('У {word1} было {cn1} {word3}. У {word2} было {cn2} {word4}.',
 'Сколько {word6} {word5}?',
 ['cn1+cn2', '', '', '', ''],
 ['', '', '', '', ''],
 {'cn1': (1, 100), 'cn2': (1, 120)},
 {'word3': 'cn1', 'word4': 'cn2'},
 {'word1': [['name_fem_gen']],
  'word2': [['name_fem_gen']],
  'word3': [['брелок', 'брелка', 'брелков', '']],
  'word4': [['word3']],
  'word5': [['word3']],
  'word6': [['у них'], ['у них всего'], ['всего'], ['у девочек всего']],
  'word7': [['']]},
 {})

In [524]:
c.format(**dv))

'В книжный супермаркет поступило 3548 экземпляров произведений Успенского. В первый день большую часть распродали и в нём осталось всего 780 экземпляров.'

In [650]:
from pymorphy2 import MorphAnalyzer, 
m = MorphAnalyzer()

In [12]:
names_masc = [line.split('\t') for line in open('dialog/names/names_masc_inf.txt').read().splitlines()]
names_fem = [line.split('\t') for line in open('dialog/names/names_fem_inf.txt').read().splitlines()]

In [663]:
names = open('dialog/names/names_fem.txt').read().splitlines()

In [657]:
[m.parse(word)[0].inflect({'accs'})[0] for word in names]

['александра',
 'андрея',
 'алексея',
 'сашу',
 'лёшу',
 'андрюшу',
 'альберта',
 'анатолия',
 'толь',
 'аркадия',
 'аркашу',
 'артёма',
 'тёма',
 'артура',
 'афанасия',
 'бориса',
 'борю',
 'вадима',
 'валентина',
 'валерия',
 'валер',
 'виктора',
 'витю',
 'виталия',
 'виталика',
 'владимира',
 'влада',
 'вову',
 'вячеслава',
 'гавриила',
 'геннадия',
 'ген',
 'глеба',
 'григория',
 'гришу',
 'даниила',
 'даню',
 'дениса',
 'дмитрия',
 'диму',
 'евгения',
 'егора',
 'ивана',
 'игоря',
 'илью',
 'илюшу',
 'кирилла',
 'кирюш',
 'кузьму',
 'левого',
 'леонида',
 'лёню',
 'максим',
 'мирона',
 'марка',
 'макса',
 'михаила',
 'мишу',
 'никиту',
 'николая',
 'колю',
 'никифора',
 'олега',
 'павла',
 'пашу',
 'петра',
 'петю',
 'прохора',
 'руслана',
 'романа',
 'рому',
 'святослава',
 'сергея',
 'серёжимого',
 'степана',
 'стёпу',
 'тараса',
 'тимофея',
 'тимошу',
 'тимура',
 'фёдора',
 'федю',
 'филиппа',
 'филю',
 'эдуарда',
 'юрия',
 'юру',
 'ярослава',
 'славу']

In [658]:
[m.parse(word)[0].inflect({'datv'})[0] for word in names]

['александру',
 'андрею',
 'алексею',
 'саше',
 'лёше',
 'андрюше',
 'альберту',
 'анатолию',
 'толю',
 'аркадию',
 'аркаше',
 'артёму',
 'тёму',
 'артуру',
 'афанасию',
 'борису',
 'боре',
 'вадиму',
 'валентину',
 'валерию',
 'валеру',
 'виктору',
 'вите',
 'виталию',
 'виталику',
 'владимиру',
 'владу',
 'вове',
 'вячеславу',
 'гавриилу',
 'геннадию',
 'гену',
 'глебу',
 'григорию',
 'грише',
 'даниилу',
 'дане',
 'денису',
 'дмитрию',
 'диме',
 'евгению',
 'егору',
 'ивану',
 'игорю',
 'илье',
 'илюше',
 'кириллу',
 'кирюшу',
 'кузьме',
 'левому',
 'леониду',
 'лёне',
 'максиму',
 'мирону',
 'марку',
 'максу',
 'михаилу',
 'мише',
 'никите',
 'николаю',
 'коле',
 'никифору',
 'олегу',
 'павлу',
 'паше',
 'петру',
 'пете',
 'прохору',
 'руслану',
 'роману',
 'роме',
 'святославу',
 'сергею',
 'серёжимому',
 'степану',
 'стёпе',
 'тарасу',
 'тимофею',
 'тимоше',
 'тимуру',
 'фёдору',
 'феде',
 'филиппу',
 'филе',
 'эдуарду',
 'юрию',
 'юре',
 'ярославу',
 'славе']

In [664]:
f = open('dialog/names/names_fem_inf.txt', 'w')
f.write('\n'.join(['\t'.join([word,
  m.parse(word)[0].inflect({'gent'})[0].capitalize(),
  m.parse(word)[0].inflect({'datv'})[0].capitalize(),
  m.parse(word)[0].inflect({'accs'})[0].capitalize()]) for word in names]))
f.close()